In [ ]:
# import tensorflow as tf
# from keras import backend as K

# num_cores = 4
# CPU = True
# GPU = False

# if GPU:
#     num_GPU = 1
#     num_CPU = 1
# if CPU:
#     num_CPU = 1
#     num_GPU = 0

# config = tf.ConfigProto(intra_op_parallelism_threads=num_cores,\
#         inter_op_parallelism_threads=num_cores, allow_soft_placement=True,\
#         device_count = {'CPU' : num_CPU, 'GPU' : num_GPU})
# session = tf.Session(config=config)
# K.set_session(session)

In [1]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import *
import matplotlib.pyplot as plt
from tokenizer import tokenizer as tweet_tokenizer
from keras.utils.np_utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers import *
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.optimizers import * 
from keras.regularizers import *
from keras.models import load_model


%matplotlib inline

PATH = './data'

train_3 = f'{PATH}/data_train_3.csv'
test_3 = f'{PATH}/data_test_3.csv'
train_7 = f'{PATH}/data_train_7.csv'
train_16m_3 = f'{PATH}/training.1600000.processed.noemoticon.csv'

Using TensorFlow backend.


## One hot sample

In [ ]:
text = 'The quick brown fox jumped over the lazy dog.'

words = set(text_to_word_sequence(text))
vocab_size = len(words)
print(vocab_size)

result = one_hot(text, round(vocab_size*1.3))
print(result)

## Tweet Tokenizer sample

In [ ]:
T = tweet_tokenizer.TweetTokenizer(preserve_case=False, preserve_url=False)

text = 'The quick brown fox jumped over the lazy dog.'

T.tokenize(text)

## Lemmatizer / Stemmer samples

In [ ]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
 
stemmer = PorterStemmer()
lemmatiser = WordNetLemmatizer()
 
print("Stem %s: %s" % ("studying", stemmer.stem("studying")))
print("Lemmatise %s: %s" % ("studying", lemmatiser.lemmatize("studying")))
print("Lemmatise %s: %s" % ("studying", lemmatiser.lemmatize("studying", pos="v")))

In [ ]:
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from collections import defaultdict
from nltk.corpus import wordnet as wn


tag_map = defaultdict(lambda: wn.NOUN, {'J': wn.ADJ, 'V': wn.VERB, 'R': wn.ADV})

s = "This is a simple sentence that would be allowing us to try lemmatizing"

tokens = word_tokenize(s)
tokens_pos = pos_tag(tokens)
 
print(tokens_pos)


lems = [lemmatiser.lemmatize(word, tag_map[pos[0]]) for word, pos in tokens_pos]
print(lems)

# Serious business

## Preprocess Train_16m_3

In [ ]:
tweets_16m = pd.read_csv(train_16m_3, sep=',', encoding='latin-1').drop(columns=['date', 'flag', 'user'])

print(tweets_16m.shape)
tweets_16m.sample(5)

In [ ]:
from TP_transfer_learning_2018 import *
from TP_transfer_learning_2018.preprocessing import standardization


tweets_16m['target'] = tweets_16m['target'].apply(lambda x: {0: 0, 2: 1, 4: 2}[x])  # Neg (0), Neu (2), Pos (4)
tweets_16m['text'] = tweets_16m['text'].apply(lambda x: standardization(x))

In [ ]:
display(tweets_16m.sample(5))
tweets_16m.sample(5)['text'].iloc[0]

In [ ]:
tweets_16m.to_feather('pickles/train_16m_3.feather')

## Preprocess Train_3

In [46]:
tweets = pd.read_csv(train_3, sep='\t', names=['ID', 'Class', 'Tweet'])

tweets.shape

(50333, 3)

In [47]:
tweets.sample(5)

,ID,Class,Tweet
17851,641558464356052992,negative,@FixTheCourt Judge rules that he can NOT issu...
25439,631629205185294338,neutral,@sonic_hedgehog Paul McCartney may have sung a...
38149,674555040636473344,positive,@Burke32_7 black Friday by Kendrick is love. T...
5182,261584813318885376,positive,Good result tonight\u002c though! Part one of ...
2650,263763678283849729,neutral,@ben_higham we put a 2nd team out and get beat...


In [48]:
from TP_transfer_learning_2018 import *
from TP_transfer_learning_2018.preprocessing import standardization


tweets['Sentiment'] = tweets['Class'].apply(lambda x: {'negative': 0, 'neutral': 1, 'positive': 2}[x])
tweets['Tweet'] = tweets['Tweet'].apply(lambda x: standardization(x))

KeyboardInterrupt: 

In [45]:
display(tweets.sample(5))
tweets.sample(5)['Tweet'].iloc[0]

,ID,Class,Tweet
519,519,0,"@BreezyWeekes hey breezy, you wanna give me so..."
831,831,-3,We're all in D. T. (Discipleship training or d...
1291,1291,1,The moon tonight 😮 #beautiful
93,93,-3,@PuseLepuru it's the irritation of not having ...
1051,1051,-2,@NessaMatthews he's perfect. But not even in t...


'Desmond was asleep when I called after I got out of my meeting... he still answered and let me vent to him the whole way home 😍 #sweetness'

In [ ]:
tweets.to_feather('pickles/train_3.feather')

## preprocess train_7

In [49]:
tweets = pd.read_csv(train_7, sep='\t', names=['ID', 'Class', 'Tweet'])

tweets.shape

(1630, 3)

In [50]:
tweets.sample(5)

,ID,Class,Tweet
751,751,0,@tunaeP_ cheer up☺️
526,526,0,it's not the most comforting &amp; jubilant pl...
1568,1568,2,"Waking up to the #scent of #coffee, there's no..."
76,76,-1,Hope was an instinct only the reasoning human ...
941,941,0,SRV's 'Voodoo Child' is approximately 76 times...


In [54]:
tweets['Sentiment'] = tweets['Class']
tweets['Tweet'] = tweets['Tweet'].apply(lambda x: standardization(x))

In [55]:
display(tweets.sample(5))
tweets.sample(5)['Tweet'].iloc[0]

,ID,Class,Tweet,Sentiment
1103,1103,1,good morning chirpy pensive sister perfect day...,1
173,173,0,everyone hat twice kinda get bop bump rn cheer,0
447,447,1,well draw lively,1
1487,1487,1,sleep past noon look window see hotel hot tub,1
822,822,-2,... nervous test rip,-2


'awesome p ok odds swap neighborhood'

In [56]:
tweets.to_feather('pickles/train_7.feather')

## Training

In [2]:
tweets = pd.read_feather('pickles/train_3.feather')

In [57]:
train_tweets, sentiments = tweets['Tweet'], tweets['Sentiment']
# train_tweets, sentiments = tweets_16m['text'], tweets_16m['target']

all_tweets = train_tweets # + test_tweets
tokenizer = Tokenizer(filters=' ')
tokenizer.fit_on_texts(all_tweets)
word_index = tokenizer.word_index

In [58]:
train_sequences = tokenizer.texts_to_sequences(train_tweets)

sequences = train_sequences # + test_sequences
MAX_SEQUENCE_LENGTH = 0
for elt in sequences:
    if len(elt) > MAX_SEQUENCE_LENGTH:
        MAX_SEQUENCE_LENGTH = len(elt)

MAX_SEQUENCE_LENGTH

28

In [5]:
train_sequences = pad_sequences(train_sequences, MAX_SEQUENCE_LENGTH)
train_sequences.shape

(50333, 32)

In [12]:
# googlenews_w2v = KeyedVectors.load_word2vec_format('data/embeddings/GoogleNews-vectors-negative300.bin', binary=True)
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import get_tmpfile


tmp_file = get_tmpfile('datastories.300d.word2vec')
glove2word2vec('data/embeddings/datastories.twitter.300d.txt', tmp_file)
w2v = KeyedVectors.load_word2vec_format(tmp_file)

In [14]:
from gensim.models import Word2Vec

w2vModel = Word2Vec.load('./data/trained_embeddings_23M.model')

In [15]:
our_w2v = w2vModel.wv
w2v = our_w2v

In [16]:
DepecheMoodpp = pd.read_csv('data/DepecheMood/DepecheMood_english_token_full.tsv', sep='\t')

print(DepecheMoodpp.loc[DepecheMoodpp['word'] == 'aa'].values.tolist()[0][1:9])

[0.0054066991014751825, 0.010635989194095686, 0.011927524123865642, 0.010163990269009415, 0.19470465335113124, 0.08262850924065103, 0.6797990668088009, 0.004733567910970831]


In [17]:
afinn = pd.read_csv('data/AFINN-111.txt', sep='\t')

def val_to_list(x):
    x += 5
    return(to_categorical(x, num_classes=11, dtype='int'))


afinn[['val']] = afinn['val'].apply(val_to_list)
print(afinn.head(10))
print(afinn.loc[afinn['word'] == 'abandon'].val)

         word                                val
0     abandon  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
1   abandoned  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
2    abandons  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
3    abducted  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
4   abduction  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
5  abductions  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
6       abhor  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
7    abhorred  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
8   abhorrent  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
9      abhors  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
0    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
Name: val, dtype: object


In [18]:
import json
with open('./data/index.json') as emojiFile:
    emoji_valence = json.load(emojiFile)

for elmt in emoji_valence:
    val = elmt['polarity']
    elmt['polarity'] = val_to_list(val)
emoji_valence

[{'emoji': '💯',
  'name': '100',
  'polarity': array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])},
 {'emoji': '😠',
  'name': 'angry',
  'polarity': array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])},
 {'emoji': '😧',
  'name': 'anguished',
  'polarity': array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])},
 {'emoji': '😲',
  'name': 'astonished',
  'polarity': array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])},
 {'emoji': '🖤',
  'name': 'black_heart',
  'polarity': array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])},
 {'emoji': '💙',
  'name': 'blue_heart',
  'polarity': array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])},
 {'emoji': '😊',
  'name': 'blush',
  'polarity': array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])},
 {'emoji': '💔',
  'name': 'broken_heart',
  'polarity': array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])},
 {'emoji': '👏',
  'name': 'clap',
  'polarity': array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])},
 {'emoji': '🤡',
  'name': 'clown_face',
  'polarity': array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])},
 {'emoji': '😰',
  'name': 'cold_sweat',
  'polarity

In [19]:
emolex = pd.read_csv('TP_transfer_learning_2018/EmoLex.txt', sep='\t')

In [20]:
positive_words = []
negative_words = []

with open('data/positive-words.txt') as positive_file, open('data/negative-words.txt', encoding='ISO-8859-1') as negative_file:
    for _ in range(35):
        next(positive_file)
        next(negative_file)
        
    for line in positive_file:
        positive_words.append(line)
    for line in negative_file:
        negative_words.append(line)

In [60]:
targets = to_categorical(sentiments, 7)
nb_words = len(word_index) + 1

EMBEDDING_DIM = 300
EMOLEX_DIM = 10
OLE_DIM = 2
EMOJI_VALENCE_DIM = 11
AFINN_DIM = 11 
DEPECHE_MOOD_DIM = 8
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM + EMOLEX_DIM + OLE_DIM + AFINN_DIM + DEPECHE_MOOD_DIM + EMOJI_VALENCE_DIM))

oov = []  # Out of vocabulary
oov.append((np.random.rand(EMBEDDING_DIM) * 2.0) - 1.0)
oov = np.full(EMBEDDING_DIM, 0)
#oov = oov / np.linalg.norm(oov)
empty_afinn = np.full(11, 0)
empty_emoji = np.full(11, 0)
empty_emolex = np.full(10, 0)
empty_depeche = np.full(8, 0)


print(empty_emolex)
print(oov.shape)

for word, i in word_index.items():
    emoji_val = 0
    
    word_vector = oov
    if word in w2v.vocab:
        word_vector = w2v.word_vec(word)

    emolex_row = emolex.loc[emolex['word'] == word]
    if emolex_row.empty:
        word_vector = np.append(word_vector, empty_emolex)
    else:
        word_vector = np.append(word_vector, emolex_row.values.tolist()[0][1:])
        
    depeche_row = DepecheMoodpp.loc[DepecheMoodpp['word'] == word]
    if depeche_row.empty:
        word_vector = np.append(word_vector, empty_depeche)
    else:
        word_vector = np.append(word_vector, depeche_row.values.tolist()[0][1:9])
        
    ole_val = [0, 0]
    if word in positive_words:
        ole_val = [1, 0]
    elif word in negative_words:
        ole_val = [0, 1]
    word_vector = np.append(word_vector, ole_val)
    
    afinn_val = empty_afinn
    emoji_val = empty_emoji
    
    afinn_row = afinn.loc[afinn['word'] == word]
    if not afinn_row.empty:
        afinn_val = afinn_row['val'].item()
    else:
        for emoji in emoji_valence:
            if word == emoji['emoji']:
                emoji_val = emoji['polarity']

    word_vector = np.append(word_vector, afinn_val)
    
    word_vector = np.append(word_vector, emoji_val)
    
    embedding_matrix[i] = word_vector

        
print(embedding_matrix.shape)

[0 0 0 0 0 0 0 0 0 0]
(300,)
(4178, 342)


In [41]:
import json
import pickle

file = open('pickles/embedding_matrix_7', 'wb')
pickle.dump(embedding_matrix, file)

In [ ]:
# Claim memory back from this very large object we don't use anymore
del w2v

In [22]:
X_train, X_val, y_train, y_val = train_test_split(train_sequences, targets, test_size=0.3)

print('training set: ' + str(len(X_train)) + ' samples')
print('validation set: ' + str(len(X_val)) + ' samples')

print('x_train:', X_train.shape)
print('y_train:', y_train.shape)

training set: 35233 samples
validation set: 15100 samples
x_train: (35233, 32)
y_train: (35233, 3)


In [23]:
def model_2017():
    vocab_size = embedding_matrix.shape[0]
    embedding_size = embedding_matrix.shape[1]

    model = Sequential()
    model.add(Embedding(vocab_size, embedding_size, weights=[embedding_matrix], 
                        input_length=MAX_SEQUENCE_LENGTH, trainable=False, name='embedding_layer'))
    model.add(GaussianNoise(0.3))
    # model.add(SpatialDropout1D(0.3))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(150, recurrent_dropout=0.3, return_sequences=True)))
    # model.add(Dropout(0.5))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(150, recurrent_dropout=0.3, return_sequences=True)))
    model.add(Dropout(0.3))
    model.add(Flatten())
    model.add(Dense(3, activity_regularizer=l2(1e-4)))
    model.add(Activation('softmax'))

    # RMSprop or lr=1e-3
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-2), metrics=['acc'])

    print(model.summary())
    
    return model

def model_mine():
    vocab_size = embedding_matrix.shape[0]
    embedding_size = embedding_matrix.shape[1]

    model = Sequential()
    model.add(Embedding(vocab_size, embedding_size, weights=[embedding_matrix], 
                        input_length=MAX_SEQUENCE_LENGTH, trainable=False, name='embedding_layer'))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(150, return_sequences=True)))
    model.add(Dropout(0.5))
    model.add(Bidirectional(LSTM(150, return_sequences=True)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

    print(model.summary())
    
    return model

In [24]:
model = model_mine()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 32, 342)           12643056  
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 342)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32, 300)           591600    
_________________________________________________________________
dropout_2 (Dropout)          (None, 32, 300)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 32, 300)           541200    
_________________________________________________________________
dropout_3 (Dropout)          (None, 32, 300)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9600)              0         
__________

In [25]:
model.fit(X_train, y_train, batch_size=128, validation_data=(X_val, y_val), epochs=5)

Train on 35233 samples, validate on 15100 samples
Epoch 1/5
35233/35233 [==============================] - 230s 7ms/step - loss: 0.8657 - acc: 0.5795 - val_loss: 0.8091 - val_acc: 0.6175
Epoch 2/5
35233/35233 [==============================] - 228s 6ms/step - loss: 0.8139 - acc: 0.6142 - val_loss: 0.7925 - val_acc: 0.6293
Epoch 3/5
35233/35233 [==============================] - 225s 6ms/step - loss: 0.7899 - acc: 0.6295 - val_loss: 0.7723 - val_acc: 0.6363
Epoch 4/5
28544/35233 [=======================>......] - ETA: 37s - loss: 0.7701 - acc: 0.6386

KeyboardInterrupt: 

In [ ]:
# model.save('pickles/train_3_66.7-emb_dr3_bi_dr5_bi_dr4.h5')

In [ ]:
model = load_model('pickles/train_3_66.7-emb_dr3_bi_dr5_bi_dr4.h5')

In [ ]:
model.fit(X_train, y_train, batch_size=128, validation_data=(X_val, y_val), epochs=10, initial_epoch=5)

In [ ]:
model.save('pickles/train_3_67.2-emb_dr3_bi_dr5_bi_dr4.h5')

In [ ]:
loss, acc = model.evaluate(X_val, y_val)

print(f'val_loss: {loss}\nval_acc: {acc}')

### Bi-derectional LSTM x2

In [ ]:
model.fit(X_train, y_train, batch_size=128, validation_data=(X_val, y_val), epochs=20)

## Transfer Learning

In [ ]:
model = load_model('pickles/train_3_67.2-emb_dr3_bi_dr5_bi_dr4.h5')

model.layers.pop()
model.layers.pop()
for layer in model.layers:
    layer.trainable = False
model.add(Dense(150, activation='relu'))
# model.add(Flatten())
model.add(Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
print(model.summary())

In [ ]:
history = model.fit(x_train_7, y_train_7, validation_data=(x_val_7,y_val_7), epochs=11, batch_size=50)

### Logistic Regression

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)

X = cv.fit_transform(tweets['Tweet'])
target = tweets['Sentiment']

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


X_train, X_val, y_train, y_val = train_test_split(X, target)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    lr = LogisticRegression(C=c, solver='saga', multi_class='auto')
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" % (c, accuracy_score(y_val, lr.predict(X_val))))

#### Best and worst features

In [ ]:
feature_to_coef = {word: coef for word, coef in zip(cv.get_feature_names(), lr.coef_[0])}

for best_positive in sorted(feature_to_coef.items(), key=lambda x: x[1])[:5]:
    print (best_positive)
    
print()
for best_negative in sorted(feature_to_coef.items(), key=lambda x: x[1], reverse=True)[:5]:
    print (best_negative)

## Learning (pseudo-code)

In [ ]:
# transfer:

model.layers.pop()
model.layers.pop()
for layer in model.layers:
    layer.trainable = False
Dense(150, 'relu')
Dense(64, 'relu')
Dense(7, 'softmax')
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

## Notes
* Télécharger 50 millions de tweets pour le word2vec
* Metric pour le script du prof: pearson
* Utiliser pearson au lieu d'accuracy
* L'année dernière le prof a eu 78% accuracy 

**Output attendu**: produire un fichier avec les mêmes noms de cartégories sur un fichier de test façon kaggle